# Teoria de Números Computacionais
## Trabalho 5: Esquema de assinatura ElGamal

#### Hugo Sousa (a76257 - LCC)
#### Matias Capitão (a82726 - LCC) 
#### Rafael Antunes (a77457 - LCC)

O esquema de assinatura ElGamal é um esquema de assinatura digital baseado no problema do logaritmo discreto e em algumas propriedades algébricas da exponenciação modular.

O algoritmo usa um par de chaves que consiste em uma **chave pública** e uma __chave privada__. A chave privada é usada para gerar uma assinatura digital para uma mensagem e essa assinatura pode ser verificada usando a chave pública correspondente do assinante.

Esta assinatura digital fornece:

- Autenticação da mensagem(O destinatário pode verificar a origem da mensagem)
- Integridade(o destinatário pode verificar se a mensagem não foi modificada desde que foi assinada)
- Não-repúdio(o remetente não pode alegar falsamente que não assinou a mensagem)


### Descrição

O esquema envolve quatro passos: geração das chaves, distribuição das chaves, assinatura e verificação.

- #### Geração das chaves

1. É escolhido um primo **p** grande(pelo menos 1024 bits)
2. Escolhe-se de seguida um $\alpha$ tal que $<\alpha>$ = $Z_p^{*}$
3. É escolhido um **x** $\in \{1,..,p-2\}$ que será a chave privada chamemos-lhe $K_{private}$
4. É computado $y = \alpha ^x (mod p)$
5. É escolhida uma função de hash criptográfica **H**

Temos agora a chave privada  $K_{private}$ e a chave pública $K_{public} = (p, \alpha, y)$

- #### Distribuição das chaves

A chave $K_{public}$ é enviada por um meio confiável, mas não nesseráriamente secreto

- #### Assinatura

Seja **m** a mensagem a ser enviada o emissor da mensagem procede da seguinte forma:

1. Escolhe um número **e** $\in\{2,...,p-2\}$ tal que $mdc(e,p-1) = 1$
2. Computa   $K_e = \alpha^e(mod p)$  a que chamamos "chave efémera" (pode ser diferente em cada comunicação)
3. O emissor por fim computa  $s = (H(m) - K_{private}*r)*e^{-1} (mod (p-1))$

A assinatura é então o par **(r,s)**

- #### Verificação

O receptor da mensagem recebe a mensagem **m** e a assinatura __(r,s)__ e com acesso à chave pública procede:

1. Verifica se $0 < r < p$ e $0 < s < p-1$
2. É computado $t = y^r * r^s $
3. Se $\alpha^{H(m)}\equiv t (mod p)$ então a assinatura é verificada caso contrário não é verificada.

### Exemplo

- __Geração das chaves__:

Escolhemos um primo, por exemplo : p = 79

Escolhemos um gerador de $Z^*_{79}$, por exemplo : $\alpha = 30$

Escolhemos um um número aleatório x $\in \{1,...,77\}$, por exemplo : x = 61

Calculamos $y = \alpha ^x (mod p)$: $y = 30^{61} (mod 79) = 59$


- __Distribuição das chaves__:

Publicamos a chave público $K_{publica} = (79, 30, 59)$ e guardamos a chave privada $K_{privada} = 61$

- __Assinatura__:

Seja a mensagem $m = 44$

O emissor escolhe um número aleatório $e \in \{2,..., 77\}$ tal que mdc(e, 78) = 1 por exemplo: e = 5

É calculada a chave efémera $K_e = 30^5 (mod 79) = 74$

É calculado $s = (44 - 61*74)*5^{-1}(mod78) = (44-4514)*47(mod 78) = -210090(mod 78) = 42$

$(74,42)$ é a assinatura e é enviada com a mensagem

- __Verificação__:

Verifica-se se  $\alpha^{m}\equiv t (mod p)$ onde $t = y^r * r^s $

v1 = $30^{44}(mod 79) = 5$


v2 = $(59^{74}*74^{42})(mod 79)= 5$

v1 = v2 Logo a assinatura é válida

### Codificação

#### Nota: 
Neste exemplo de código vamos assumir que a função de hash criptográfica está definida e vamos usar número relativamente pequenos para meios de exemplificação.

In [11]:
import random

class ElGamal_DS:
    def __init__(self):
        p, g, x, y = self.gen_keys()
        self.p = p
        self.g = g
        self.private = x
        self.public = y
    
    @staticmethod   
    def gen_keys(nbits = 16):
        #é escolhido um nr primo aleatório
        p = random_prime(2^(nbits//2), 2^(nbits//2-2))
        #é escolhido um gerador(raiz primitiva) do grupo multiplicativo
        g = primitive_root(p)
        #é escolhido um nr x que será a chave privada
        x = randint(1,p-1)
        #é computada a chave pública
        y = power_mod(g,x,p)
        return p, g, x, y
    
    def publish_key(self):
        return(self.p, self.g, self.public)
    
    def sign(self, m):
        e = randint(2, self.p-2)
        while(gcd(e,(self.p -1))!=1):
            e = randint(2,(self.p-2))
        r = power_mod(self.g, e, self.p)
        aux1 = m-(self.private *r)
        aux2 = power_mod(e, -1, self.p - 1)
        s = mod(aux1*aux2,self.p-1)
        return(r,s)
    
    def verify(self, msg, sign, pubk):
        #verifica se 0<r<p e  0<s<p-1
        if(not(0 < sign[0] < pubk[0]) and not(0 < sign[1] < (pubk[0] - 1))):
            print("Signature not valid!")
            return
        t = mod(pow(pubk[2],sign[0]) * pow(sign[0], sign[1]), pubk[0])
        m = power_mod(pubk[1], msg, pubk[0])
        if(t == m):
            print("Assinatura verificada! Mensagen é autêntica")
        else:
            print("Erro de assinatura!! Mensagem não é autêntica!")

### Exemplo

Bob manda mensagem assinada a Alice

In [12]:
#Bob gera as suas chaves 
Bob = ElGamal_DS()
msg =123

#Bob publica a chave publica
pkey = Bob.publish_key()

#Bob assina a msg e envia a Alice
signature= Bob.sign(msg)

In [13]:
Alice = ElGamal_DS()

#Alice recebe a mensagem verifica a assinatura
Alice.verify(msg, signature , pkey)

Assinatura verificada! Mensagen é autêntica


Chuck tenta fazer-se passar por Bob

In [14]:
Chuck = ElGamal_DS()

#Chuck intercepta a mensagem de Bob assina-a e manda a Alice
false_sign = Chuck.sign(msg)

In [15]:
Alice = ElGamal_DS()

#Alice recebe a mensagem  de Chuck verifica a assinatura
Alice.verify(msg, false_sign , pkey)

Erro de assinatura!! Mensagem não é autêntica!


### Bibliografia

David R. Kohel - Cryptography

<https://en.wikipedia.org/wiki/ElGamal_signature_scheme> - Wikipedia: Elgamal Signature Scheme

<https://www.youtube.com/watch?v=iiukwTar6Fo&list=PL1xkDS1G9As4Yz_te20j1A9evIjt5Z06e&index=128> - Applied Cryptography: The ElGamal Digital Signature